In [53]:
import pickle
import re
import numpy as np

sequenceLength = 15
data_dir = '../data/kanye_verses.txt'
data_dir2 = '../data/kanye_new.txt'

In [54]:
file = open("bannedwords", 'rb')
bannedwords = pickle.load(file)
file.close()

file = open("prereplacements", 'rb')
replacements = pickle.load(file)
file.close()

file = open("postrep", 'rb')
postreplacements = pickle.load(file)
file.close()

file = open("charstoclean", 'rb')
charactersToClean = pickle.load(file)
file.close()

file = open("charstospace", 'rb')
charactersToSpace = pickle.load(file)
file.close()

In [55]:
def cleanLine(text):
    for x, y in replacements:
        text = text.lower()
        if x in text:
            text = text.replace(x, y)
    
    for char in charactersToClean:
        text = text.replace(char, "")

    for char in charactersToSpace:
        text = text.replace(char, " ")
 
    text = re.sub(r'[0-9]+', '', text)
    
    for x, y in postreplacements:
        text = text.lower()
        if x in text:
            text = text.replace(x, y)      
    return text

def removeLRWhitespace(line):
    x = line.lstrip().rstrip()
    return x

In [56]:
def loadDataLines(loadboth=True):
    if(loadboth):
        file1 = open(data_dir2, 'r')
        text1 = file1.read()
        text1 = text1[43:]
        text1 = text1[:-6]
        text1 = text1.lower().split("\\n")
        lines1 = [cleanLine(line) for line in text1]
        lines1 = [removeLRWhitespace(line) for line in lines1]
    
    file2 = open(data_dir, 'r')
    text2 = file2.read()
    text2 = text2.replace("\n\n","\n")
    text2 = text2.lower().split("\n")
    lines2 = [cleanLine(line) for line in text2]
    lines2 = [removeLRWhitespace(line) for line in lines2]
    
    if(loadboth):
        lines = lines1+lines2
    else:
        lines = lines2
        
    lines.remove("")
    return lines

In [57]:
def removeBannedWords(lines, bannedwords):
    mostused = {}
    temp = []
    bannedcount=0
    for line in lines:
        banned = False
        for word in bannedwords:
            if word in line:
                try:
                    val = mostused[word]
                    mostused[word] = val+1
                except KeyError:
                    mostused[word] = 1
                    
                banned = True
                bannedcount+=1
                break
        if(banned == False):
            temp.append(line)
    return temp, mostused, bannedcount

In [58]:
def slidingWindow(line, sentenceLength):
    sentences = []

    if len(line.split()) > sentenceLength:
        for i in range(sentenceLength, len(line.split())):
            seq = line.split()[i-sentenceLength:i+1]
            sentences.append(" ".join(seq))
        return sentences
    else:
        return [line]
    
lines = loadDataLines(loadboth=True)

temp = []
for line in lines:
    sos="<|sos|> "
    eos=" <|eos|>"
    newline = sos+line+eos
    temp.append(newline)
lines = temp

augmentedLines = [slidingWindow(line, sequenceLength) for line in lines]
#augmentedLines = lines

temp = []
for group in augmentedLines:
  if(len(group)!=0):
    temp.append(group)

augmentedLines = temp

augmentedLines = sum(augmentedLines, [])
augmentedLines = np.unique(np.array(augmentedLines)).tolist()

In [59]:
final, dic, linecount = removeBannedWords(augmentedLines, bannedwords)

In [60]:
final

['<|sos|>  <|eos|>',
 '<|sos|> a  dollar fade every single time you see us <|eos|>',
 '<|sos|> a  gauge stashed in the back right beside the speaker <|eos|>',
 '<|sos|> a  keys at the piano <|eos|>',
 '<|sos|> a  year old  year old with rich nigga problems <|eos|>',
 '<|sos|> a <|eos|>',
 '<|sos|> a a a j <|eos|>',
 '<|sos|> a a and you be givin me my space a a and <|eos|>',
 '<|sos|> a a p t r a p l o r d nigga pay me <|eos|>',
 '<|sos|> a a tall light skinned dude with a beard and shit <|eos|>',
 '<|sos|> a b c d e f g someone shouldve told you not to fuck with',
 '<|sos|> a baby aint temporary that shits forever ma <|eos|>',
 '<|sos|> a backpack nigga with luxury taste buds <|eos|>',
 '<|sos|> a backpack nigga with luxury tastebuds <|eos|>',
 '<|sos|> a bad bitch and a bag of weed the holy trinity <|eos|>',
 '<|sos|> a bad bitch with hips <|eos|>',
 '<|sos|> a bat in the wind cant see me until june like <|eos|>',
 '<|sos|> a battle the child wont know about <|eos|>',
 '<|sos|> a bea

In [8]:
temp=[]
for line in final:
    linelength = len(line.split())
    paddingamount = (sequenceLength+1)-linelength
    if(paddingamount > 0):
        newline = line.split()
        for i in range(paddingamount):
            newline.append("<|eos|>")
        temp.append(" ".join(newline))
    else:
        temp.append(line)    
withpad = temp

In [9]:
dataname="preprocessedLinesSparse"
file = open(dataname, 'wb')
pickle.dump(withpad, file)
file.close()